In [ ]:
pip install sqlalchemy

In [ ]:
pip install snowflake-sqlalchemy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 51.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 72.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 7.3 MB/s eta 0:00:00
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.27
    Uninstalling SQLAlchemy-2.0.27:
      Successfully uninstalled SQLAlchemy-2.0.27
  Attempting uninstall: platformdirs
    Found existing installation: platformdirs 4.2.0
    Uninstalling platformdirs-4.2.0:
      Successfully uninstalled platformdirs-4.2.0
  Attempting uninstall: cryptography
    Found existing installation: cryptography 42.0.2
    Uninstalling cryptography-42.0.2:
      Successfully uninstalled cryptography-42.0.2
  Attempting uninstall: pyOpenSSL
    Found existing installation: pyOpenS

In [ ]:
pip install --upgrade snowflake-sqlalchemy

# Documentation

## Snowflake Database Setup Script

### Description:
This script automates the setup process for a Snowflake database, including creating a database, tables, warehouse, and uploading data from a CSV file.

### Usage:
1. Make sure you have the necessary credentials and access to a Snowflake account.
2. Update the `user`, `password`, `account_identifier`, and `database_name` variables with your Snowflake account details.
3. Execute the script using Python.

### Dependencies:
- Python 3.x
- SQLAlchemy
- Snowflake Connector

### Script Steps:
1. **Snowflake Connection Setup:**
   - Establishes a connection to the Snowflake database using SQLAlchemy.

2. **Database Creation:**
   - Creates or replaces the specified database.

3. **Table Creation:**
   - Creates or replaces the `reading_details` table with specific column definitions.

4. **Warehouse Creation:**
   - Creates or replaces a warehouse with predefined settings.

5. **Stage Creation and Data Upload:**
   - Creates a stage for data loading.
   - Uploads the CSV file (`refresher_readings.csv`) to the specified stage.

6. **Data Loading into Table:**
   - Copies the data from the stage to the `reading_details` table.

7. **Cleanup:**
   - Disposes the SQLAlchemy engine to release resources.

### Usage Example:
```bash
python snowflake_setup_script.py


Note:
Ensure that the CSV file (refresher_readings.csv) is present in the specified directory (file:///content/) before executing the script.
Modify the file paths and Snowflake settings as needed for your environment.

In [8]:
#!/usr/bin/env python
import warnings
from sqlalchemy import create_engine

warnings.filterwarnings("ignore")

def main():
    user = 'SudarshanDudhe'
    password = 'SDudhe123'
    account_identifier = 'ofewoso-db39110'
    database_name = 'rr_data'

    engine = create_engine(
        f"snowflake://{user}:{password}@{account_identifier}/"
    )

    with engine.connect() as connection:
        # Create database
        connection.execute(f"CREATE OR REPLACE DATABASE {database_name};")

        # Create table
        connection.execute("""CREATE OR REPLACE TABLE reading_details (
            title STRING,
            topic STRING,
            year STRING,
            level STRING,
            introduction STRING,
            learning_outcomes STRING,
            summary STRING,
            overview STRING
            )""")

        # Create warehouse
        connection.execute("""CREATE OR REPLACE WAREHOUSE rr_data_wh WITH
           WAREHOUSE_SIZE='X-SMALL'
           AUTO_SUSPEND = 180
           AUTO_RESUME = TRUE
           INITIALLY_SUSPENDED=TRUE;
           """)

        # Create stage and upload files
        connection.execute("USE DATABASE rr_data")
        connection.execute("""CREATE STAGE REFRESHER_READINGS DIRECTORY = ( ENABLE = true );""")
        connection.execute("""PUT file:///content/refresher_readings.csv @rr_data.public.refresher_readings;""")

        # Copy stage to table
        connection.execute("USE WAREHOUSE rr_data_wh")
        connection.execute("""COPY INTO reading_details
          FROM @rr_data.public.refresher_readings
          FILE_FORMAT = (type = csv field_optionally_enclosed_by='"')
          PATTERN = '.*refresher_readings.csv.gz'
          ON_ERROR = 'skip_file';""")

    print("Done")
    engine.dispose()

if __name__ == "__main__":
    main()


Done
